Target:
* Add dropout regularization with a probability of 10% after each layer (except the final layer) to reduce overfitting

Result:
* Max Train accuracy (in 15 epochs): 98.67%.
* Max test accuracy (in 15 epochs): 99%

Analysis:
*  The dropout has reduced the accuracy of both train and test which is expected. However the test accuracy is generally higher than train accuracy.
* I need to boost my trainset by adding data transformations and augmentations. I will include image rotation in the next iteration to try and improve train accuracy

In [ ]:
!git clone "https://github.com/jyanivaddi/ERA_V1.git"
!git pull

Add all the imports

In [ ]:
from __future__ import print_function
import sys
sys.path.append("ERA_V1/session_7")
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import transforms
from s7_utils import load_mnist_data, preview_batch_images, plot_statistics
from s7_model import Model_6_Net, model_summary, model_train, model_test

Allocate GPU

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
mnist_model = Model_6_Net().to(device)
model_summary(mnist_model, input_size=(1,28,28))

Define Transforms

In [ ]:
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])
test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

Download Data

In [ ]:
train_data, test_data = load_mnist_data(train_transforms, test_transforms)

Define train and test loaders

In [ ]:
torch.manual_seed(1)
batch_size = 128
kwargs = {'num_workers': 2, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle = True, **kwargs)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = batch_size, shuffle = True, **kwargs)

Preview data

In [ ]:
preview_batch_images(train_loader)

In [ ]:
model = Model_6_Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1, verbose=True)
train_losses = []
test_losses = []
train_acc = []
test_acc = []
for epoch in range(1,20):
    print(f"epoch: {epoch}")
    model_train(model, device, train_loader, optimizer, train_acc, train_losses)
    model_test(model, device, test_loader, test_acc, test_losses)
    scheduler.step()


Plot Statistics

In [ ]:
plot_statistics(train_losses, train_acc, test_losses, test_acc)